In [1]:
from Bio import SeqIO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

In [2]:
import sys
sys.path.append('../../code')
import visualize
from crispr import CasTemplate
from pipeline import NDNFPipeline

In [3]:
cas_templates = [CasTemplate(pam_motif = 'NNGRRT', typ='II', 
                             protospacer_length=20, name='SaCas9-WT'),
                 CasTemplate(pam_motif = 'NGG', typ='II', 
                             protospacer_length=20, name='SpCas9-WT'),
                 CasTemplate(pam_motif = 'TTTN', typ='V',
                             protospacer_length=20, name='LbCas12a_Cpf1')
                 ]                 

In [4]:
# Path to a collection of HIV sequences to use as the nomination set.
NOMINATE_FASTA = '../../data/nominate.fasta'

# Path to a collection of HIV sequences to use as the validation set.
VALIDATE_FASTA = '../../data/test.fasta'

# Path to a single HIV sequence to use as the common reference position set.
REFERENCE_FASTA = '../../data/hxb2.fasta'

# Path to an off-target reference genome.
OFFTARGET_REF = '../../data/GRCh38.fna'

# The number of protospacers to extract from the nomination set.
TOP_N_KMERS = 20_000

# The target number of protospacers after diversification.
DIVERSITY_TARGET = TOP_N_KMERS*2

MISMATCH = 2
BROAD_SPECTRUM = 0.9
MAX_OFF = 0



In [5]:
with open(NOMINATE_FASTA) as handle:
    nomination_seqs = list(SeqIO.parse(handle, 'fasta'))
num_nom = len(nomination_seqs)
len_nom = sum(len(seqR.seq) for seqR in nomination_seqs)/1_000_000
print(f'Loading {num_nom} sequences with a total of {len_nom:0.2f} Mbp for nomination.')

Loading 3228 sequences with a total of 28.73 Mbp for nomination.


In [6]:
MUT_DATA = pd.read_csv('../../data/mutation_scores.csv')

def rc_scores(start, width=20):
    
    mask = (MUT_DATA['Position'] <= (start+width)) & (MUT_DATA['Position'] >= (start-width))
    mean_rc = MUT_DATA.loc[mask, 'RC Index'].mean()
    
    return mean_rc

def lethal_count(start, width=20):
    
    mask = (MUT_DATA['Position'] <= (start+width)) & (MUT_DATA['Position'] >= (start-width))
    num_lethal = (MUT_DATA.loc[mask, 'Effect'] == "lethal").sum()
    
    return num_lethal

def process_cas(cas, mismatch, protospacer_counts, diversed, ref_diversed):
    
    narrowed, protospacer_hist_freq = diversed.narrow(VALIDATE_FASTA, min_rate=0.5, return_freqs=True,
                                                      mismatch=mismatch)
    good_hits, off_target_counts = narrowed.filter(OFFTARGET_REF, 
                                               mismatch = mismatch, 
                                               return_counts=True)
    
    ref_diversed['on_target_freq'] = ref_diversed['protospacer'].map(protospacer_hist_freq.get)
    ref_diversed['off_target_hits'] = ref_diversed['protospacer'].map(off_target_counts.get)
    ref_diversed['mismatch'] = mismatch
    return ref_diversed
    

In [7]:
for cas in cas_templates:
    print('Processing', cas.name)
    # Pre-calculate the things independent of mismatch value
    nominated, protospacer_counts = NDNFPipeline.nominate(cas, nomination_seqs, 
                                                          top_n=TOP_N_KMERS, 
                                                          return_counts=True)
    diversed = nominated.diversify(DIVERSITY_TARGET)
    
    ref_diversed = diversed.reference_map(REFERENCE_FASTA, mismatch=3, enforce_pam=False)
    ref_diversed['num_lethal'] = ref_diversed['start'].map(lethal_count)
    ref_diversed['mean_rc'] = ref_diversed['start'].map(rc_scores)
    ref_diversed['cas'] = cas.name
    ref_diversed['pam_motif'] = cas.pam_motif
    
    for nmiss in [0, 1, 2, 3, 4]:
        print('Processing', cas.name, nmiss)
        fname = f'{cas.name}.{nmiss}.info.csv'
        if not os.path.exists(fname):
            process_cas(cas, nmiss, protospacer_counts, diversed, ref_diversed.copy()).to_csv(fname, index=False)

Processing SaCas9-WT


100%|██████████| 40000/40000 [07:03<00:00, 94.53it/s]    
Total 1 device(s) found.
Loading input file...
Reading ../../data/hxb2.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
35.9129 seconds elapsed.


Processing SaCas9-WT 0


Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
47.6974 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
31.7711 seconds elapsed.


Processing SaCas9-WT 1


...

Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
72.9805 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
156.607 seconds elapsed.


Processing LbCas12a_Cpf1 3


Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
91.7136 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
391.107 seconds elapsed.


Processing LbCas12a_Cpf1 4


Total 1 device(s) found.
Loading input file...
Reading ../../data/test.fasta...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
107.716 seconds elapsed.
Total 1 device(s) found.
Loading input file...
Reading ../../data/GRCh38.fna...
Sending data to devices...
Chunk load started.
1 devices selected to analyze...
Finding pattern in chunk #1...
Comparing patterns in chunk #1...
1 devices selected to analyze...
Finding pattern in chunk #2...
Comparing patterns in chunk #2...
686.13 seconds elapsed.
